In [1]:
import numpy as np
import pandas as pd
import os
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras import callbacks 
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [2]:
!git clone 'https://github.com/anirudhv14/SDCNL.git'

Cloning into 'SDCNL'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 63 (delta 27), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (63/63), 1.87 MiB | 1.54 MiB/s, done.


In [3]:
!ls './SDCNL/data'

combined-set.csv    processed_train.csv  training-set.csv
processed_test.csv  testing-set.csv


In [4]:
data_train = pd.read_csv('./SDCNL/data/training-set.csv',usecols=["selftext","is_suicide"])
data_test =  pd.read_csv('./SDCNL/data/testing-set.csv',usecols=["selftext","is_suicide"] )
# data_train.head()

data_train.rename(columns={"selftext":"text","is_suicide":"class"},inplace=True)
data_test.rename(columns={"selftext":"text","is_suicide":"class"},inplace=True)
# data_test.head()

In [5]:
def change_label(value):
    if value =="SuicideWatch":
        return 1
    else :
        return 0
df = pd.read_csv("../input/suic-mini-proj/SuicideAndDepression_Detection.csv").dropna()
df.loc[df['class'].isin(['SuicideWatch', 'depression'])]
# df.drop("Unnamed: 0",axis=1,inplace=True)
df['class']=df['class'].apply(change_label)
df.head()

,text,class
0,Does life actually work for most / non-depress...,0
1,I found my friend's bodyIt was almost nine yea...,0
2,Ex Wife Threatening SuicideRecently I left my ...,1
3,Am I weird I don't get affected by compliments...,0
4,Finally 2020 is almost over... So I can never ...,0


In [6]:
df_new=pd.concat([data_train,data_test,df])
df_new.rename(columns={"class":"label"},inplace=True)

In [7]:
df_new

,text,label
0,Hi I don't really know how to phrase this situ...,0
1,i have been so depressed these past couple wee...,1
2,Hi..I don't know where else to go. I am devast...,0
3,"The shit like “it will get better, everyone is...",1
4,I wish I was prettier. I wish I didn’t feel li...,0
...,...,...
348119,You how you can tell i have so many friends an...,0
348120,pee probably tastes like salty tea😏💦‼️ can som...,0
348121,The usual stuff you find hereI'm not posting t...,1
348122,"I confronted my mother. Extremely isolated, wi...",0


In [8]:
print("Suicide:\n")
for ctr,i in enumerate(df_new.text[df_new.label == 1]):
    print(ctr,':',i,'\n')
    if(ctr > 2):
        break
print("\n\n\nDepression:\n")
for ctr,i in enumerate(df_new.text[df_new.label == 0]):
    print(ctr,':',i,'\n')
    if(ctr > 2):
        break

Suicide:

0 : i have been so depressed these past couple weeks ever since i got back from vacation normal life and reality has hit me and its terrible. im 19 i havent moved out yet i dont really have a sufficient job i have one but its not enough money for me to support myself. at least i have a car and im taking online classes so. i feel stuck in this stupid fucking town. empty and alone man i just want someone to love me dude but im ugly and fat and worthless and no one is ever going to see im worth the chance. its just one thing after another to worry about. why not end it now before i really fuck up and become miserable? i dont see the point in going on im just really down right now i dont know what to do )): all there is darkness idk how to get out of this hole i hate myself i hate my body i hate my life i dont think it will ever get better 

1 : The shit like “it will get better, everyone is here for a purpose you just need to find yours.” Or “it will get better just wait, one da

In [9]:
# print(data.selftext.iloc[0], '\n--------\n', data.selftext_clean.iloc[0])

In [10]:
# print(len(set(data.title)), len(data))
# print(len(set(data.title_clean)), len(data))

In [11]:
# df = data_train[['selftext', 'is_suicide']].dropna()#.drop(list(set(data.columns) - set(['selftext_clean', 'is_suicide'])), axis =  1).dropna()
# df.head()

In [12]:
import collections
collections.Counter(df_new.label)

Counter({0: 232987, 1: 117017})

In [13]:
# import collections

# aa = collections.Counter(df.num_comments)

In [14]:
# plt.bar(aa.keys(), aa.values())

In [15]:
# print("Average sentance length:", str(np.mean([len(df.selftext_clean.iloc[i].split(' ')) for i in range(len(df))])) + ' words')

# a1 = []
# for k in [j for j in [df.selftext_clean.iloc[i].split(' ') for i in range(len(df))]]:
#     a1.extend(k)
# print("Average word length:", str(np.mean([len(i) for i in a1]))+' letters')

In [16]:
print("Average sentance length:", str(np.mean([len(df_new.text.iloc[i].split(' ')) for i in range(len(df_new))])) + ' words')

a1 = []
for k in [j for j in [df_new.text.iloc[i].split(' ') for i in range(len(df_new))]]:
    a1.extend(k)
print("Average word length:", str(np.mean([len(i) for i in a1]))+' letters')

Average sentance length: 170.99887715568966 words
Average word length: 4.253366002848675 letters


In [17]:
# avg = sum([i for i in data.num_comments.values])//len([i for i in data.num_comments.values])
# print('Least number of comments for a post: {} \nMost number of comments for a post: {} \nAverage comments per post: {}'.format(min(data['num_comments'].values),max(data['num_comments'].values), avg))

In [18]:
!pip install scattertext

In [19]:
data1 = df_new.copy().sample(100000)
a1 = []
for i in data1.label.values:
    if(i == 0):
        a1.append('depression')
    else:
        a1.append('suicide')
data1['label'] = a1

In [20]:
import scattertext as st
import spacy
from pprint import pprint

nlp = spacy.load('en_core_web_sm')
corpus = st.CorpusFromPandas(data1, 
                              category_col='label', 
                              text_col='text',
                              nlp=nlp).build()


In [21]:
print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

['myself', 'filler', 'depression', 'depressed', 'anymore', 'feel', 'shit', 'suicide', 'wo', 'suicidal']


In [22]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['suicide'] = corpus.get_scaled_f_scores('suicide')
pprint(list(term_freq_df.sort_values(by='suicide', ascending=False).index[:10]))

['end it',
 'bye',
 'n’t want',
 'to end',
 'kill myself',
 'to die',
 'ca n’t',
 'to kill',
 'killing myself',
 'kill']


In [23]:
term_freq_df['depression'] = corpus.get_scaled_f_scores('depression')
pprint(list(term_freq_df.sort_values(by='depression', ascending=False).index[:50]))

['\u200d \u200d',
 '⣿',
 '⠀',
 '⣿ ⣿',
 '⠀ ⠀',
 'filler filler',
 '⠄',
 '⠄ ⠄',
 'ni ni',
 'cecil',
 'cecil cecil',
 'filler',
 '\u200d',
 'ni',
 'ㅤ',
 '\u200e\u200f\u200f\u200e',
 'fuck fuck',
 '\u200e\u200f\u200f\u200e \u200e\u200f\u200f\u200e',
 '#',
 '█',
 '😎',
 '.-',
 '️',
 '_ _',
 'u',
 '_',
 'bored',
 'do you',
 'my depression',
 'depression',
 'hey',
 'guys',
 '😎 😎',
 'how do',
 'does anyone',
 'depression is',
 'with depression',
 'crush',
 '︎',
 '︎ ︎',
 'of depression',
 'game',
 'play',
 'class',
 'girls',
 'anyone else',
 'you guys',
 '/',
 'advice',
 'games']


In [24]:
html = st.produce_scattertext_explorer(corpus,
          category='suicide',
          category_name='Suicide',
          not_category_name='Depression',
          width_in_pixels=1000)

open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

103693688

In [25]:
# dispersion = st.Dispersion(corpus)

# dispersion_df = dispersion.get_df()
# dispersion_df.head(3)

In [26]:
# dispersion_df = dispersion_df.assign(
#     X=lambda df: df.Frequency,
#     Xpos=lambda df: st.Scalers.log_scale(df.X),
#     Y=lambda df: df["Rosengren's S"],
#     Ypos=lambda df: st.Scalers.scale(df.Y),
# )

In [27]:
# from sklearn.neighbors import KNeighborsRegressor

# dispersion_df = dispersion_df.assign(
#     Expected=lambda df: KNeighborsRegressor(n_neighbors=10).fit(
#         df.X.values.reshape(-1, 1), df.Y
#     ).predict(df.X.values.reshape(-1, 1)),
#     Residual=lambda df: df.Y - df.Expected,
#     ColorScore=lambda df: st.Scalers.scale_center_zero_abs(df.Residual)
# ) 

In [28]:
# html = st.dataframe_scattertext(
#     corpus,
#     plot_df=dispersion_df, 
#     metadata=corpus.get_df()['author'] + ' (' + corpus.get_df()['is_suicide'].str.upper() + ')',
#     ignore_categories=True,
#     x_label='Log Frequency',
#     y_label="Rosengren's S",
#     y_axis_labels=['More Dispersion', 'Medium', 'Less Dispersion'],
#     color_score_column='ColorScore',
#     header_names={'upper': 'Lower than Expected', 'lower': 'More than Expected'},
#     left_list_column='Residual',
#     background_color='#e5e5e3'
# )

# open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))